# sc3nb – SuperCollider3 for python / jupyter notebooks
&copy; 2017-2020 by Thomas Hermann, contributions by: Ferdinand Schlatt, Fabian Kaupmann, Dennis Reinsch, Micha Steffen Vosse
* * *

sc3nb is a python module to enable interaction with and control of SuperCollider3 platform from python, and specifically from ipython notebooks resp. jupyter notebooks.

The goal of sc3nb is to facilitate the programming of auditory displays and interactive sonifications by
- teaming up python (and particularly numpy, scipy, pandas, matplotlib etc.) for data science 
- and SuperCollider3 for interactive real-time sound rendering.

It currently features
- The SuperCollider audio server can be started and addressed via a python SuperCollider Server implementation or via OSC messages directly.
- There are many Python implementations of Objects from SuperCollider like `Synth`, `Group` or `Buffer`
- Also the SuperCollider Language can be used interactively via a subprocess

It is meant to grow into a backend for a sonification package, and can be used both from jupyter and in standard python software development.

sc3nb is hosted at GitHub https://github.com/thomas-hermann/sc3nb

## Getting Started

In [ ]:
# setup, heading
import time, random, os

import numpy as np

%matplotlib inline
import matplotlib.pyplot as plt

In [ ]:
import sc3nb as scn

To startup sc3nb (sclang, scsynth, and a python OSC server) use `startup`
* sclang and scsynth are started as subprocesses and thier outputs is collected. 
* If the executables are not in `$PATH`, you can specify them with `sclang_path`/`scsynth_path` f.e. `sclang_path="/path/to/sclang-containing-dir/sclang"`
    * On Mac they reside in /Applications/SuperCollider.app/Contents in the folders MacOS and Resources
      to add these paths to your `$PATH`, simply add to your ~/.profile, e.g. (please adapt to your installation):
    `PATH=$PATH:/Applications/SuperCollider.app/Contents/MacOS:/Applications/SuperCollider.app/Contents/Resources`

In [ ]:
sc = scn.startup()  # optionally specify arg sclang_path="/path/to/sclang"

## Caveats and Problems

We encourage you to share any problems that arise using sc3nb with us.

There are some things to consider when using sc3nb.

- The shortcut `ctrl/cmd + .` does currently only works in classic Jupyter notebooks not in JupyterLab. It also will import sc3nb and thus you should avoid variables with the name sc3nb.
- We depend on the Output to stdout of sclang and scsynth in some cases like the startup of the server or more obviously for the sclang output when using cmd with verbosity. This means if some language settings or updates of sclang/scsynth change the outputs to something that we don't expect stuff will fail.  
- There is a bug in Jupyter (https://github.com/jupyter/jupyter_client/issues/104) that does leave sclang and scsynth running when restarting the Jupyter kernel. However calling `exit()` on the SC instance will kill them. To aviod conflicts with orphaned sclangs/scsynths we also look for leftover sclang/scsynth processes on the start of each and will try to kill them. This might lead to killing sclang or scsynth processes that you wanted to keep. You can specify which parent processes are allowed for sclang/scsynth processes with the `allowed_parents` paramter.

## sclang

You can send commands and receive data directly from the SuperCollider Language

### sclang command execution

To send sc3 commands (i.e. program strings) to the language, either use the functions
* **`cmd()`** normal command sending.
* **`cmdv()`** verbose, i.e. returning string is collected and output to notebook ( alias for `cmd(.., verbose=True)` )
* **`cmdg()`** send program and get and parse the output ( alias for `cmd(.., get_return=True)` )

In [ ]:
# sc.cmd(cmdstr, pyvars)
sc.lang.cmd('"hello".postln')  # check jupyter console for output

use cmdv If sclang output should be displayed as cell output

In [ ]:
sc.lang.cmdv('"sc3nb".postln')  # also check jupyter console for output

or use the corresponding Magics in Jupyter

* Jupyter line magics **%sc, %scv, %scg, %scgv**
* Jupyter cell magics **%%sc, %%scv, %%scg, %%scgv**

which wrap the above functions. See examples below

Cell magics can be placed within code just as the function calls as shown here:

In [ ]:
%sc x = Synth.new(\default, [\freq, 100])
for p in range(1, 10):  # a bouncing ball
    %sc Synth.new(\s1, [\freq, 200])  // this is sc cell so use sc3 comments instead of #
    time.sleep(1/p)
# NOTE: Windows needs delay between sclang inputs, so use sleep

%sc x.free 

Use raw python strings for multiline sc3-programs:

In [ ]:
sc.lang.cmd(r"""
Routine({
    x = 10.collect{ |i|
        0.2.wait;
        Synth.new(\default, [\freq, 50+(50*i)]);
    };
    1.wait;
    x.do{|e| 
        e.release;
        0.1.wait;};
}).play;
""")

alternatively, you can use the cell magic:

In [ ]:
%%sc
Routine({
    x = 5.collect{ |i|
        0.2.wait;
        Synth.new(\default, [\freq, 50+(50*i)]);
    };
    1.wait;
    x.do{|e| 
        e.release;
        0.1.wait;};
}).play;

Note that the code is executed in sclang and python is returning directly after sending the command.

### sclang command execution with python variable injection

Python variables can be injected into sc3 commands by using the ^ special: The following examples demonstrates it by setting frequencies by using python variables

In [ ]:
for p in range(1, 50):  # a tone ladder
    freq = 50 + p*3
    dur = np.log(p)
    position = np.sign(p-25)
    %sc Synth.new(\s1, [\freq, ^freq, \dur, ^dur, \pan, ^position])
    time.sleep(0.05)

This is injection is done with

In [ ]:
scn.sclang.convert_to_sc?

In [ ]:
python_list = [1,2,3,4]
%scv ^python_list.class

In [ ]:
complex_py = 1+1j
%scv ^complex_py.class

In [ ]:
symbol = r"\\python"
%scv ^symbol.class

When using the `cmd` | `cmdv` | `cmdg` functions you can also provide a dictionary with variable names as keys and content as values (which can use other python vars or statements)

In [ ]:
sc.lang.cmdv("^name1 / ^name2", pyvars={'name1': 9,'name2': 9*2})

In [ ]:
# without providing pyvars, variables are searched in the users namespace.
freq = 5
rate = 6
sc.lang.cmdv("(^freq + 1) * (^rate + 1)")

In [ ]:
# alternatively via the magic this is done as:
%scv (^freq + 1) * (^rate + 1)

### Getting sclang output in python

* To get the output of an sclang snippet into a python variable, use the cmdg function.
* The following example shows how to transfer a synth's nodeID

In [ ]:
# start a Synth
sc.lang.cmd(r"""x = Synth.new(\default)""")

In [ ]:
# get the nodeId to python
nodeID = sc.lang.cmdg("x.nodeID")
print(nodeID)

In [ ]:
# use the nodeID to free the Synth via a message to scsynth audio server directly
sc.server.msg("/n_free", nodeID)  

**sc.cmdg(), resp. %scg return integers, floats, strings and lists**
* %scg can be assigned to a python variable within code

In [ ]:
a = %scg 1234 + 23452
print(f"returned an {type(a)} of value {a}")

In [ ]:
a = %scg 1234.5.squared
print(f"returned an {type(a)} of value {a}")

In [ ]:
%scg "sonification".scramble

be careful when using magics directly after another

In [ ]:
%sc ~retval = "sonification".scramble
time.sleep(0.001)  # (in Windows) without pause an Empty might be thrown..
%scg ~retval

(in Windows) it should be prefered to combine your code in one cmd execution.

In [ ]:
scramble = %scg ~retval = "sonification".scramble; ~retval ++ "!";
scramble

In [ ]:
a = %scg (1,1.1..2)

In [ ]:
a

In [ ]:
a = %scg (1,1.1..2)
print(f"list with length: {len(a)}")
a

Note that floating-point numbers do have a limited precision

In [ ]:
[round(num, 6) for num in a]

However they should be close

In [ ]:
np.allclose(a, np.arange(1, 2, 0.1))

### Cell magics

In [ ]:
%sc {SinOsc.ar(MouseX.kr(200,400))}.play  // move mouse horizontally, CMD-. to stop

In [ ]:
%sc s.scope()

In [ ]:
%sc s.freeAll

In [ ]:
value = %scg 75-25
print("value = ", value)

In [ ]:
%%sc
{
    x = Synth.new(\s2, [\freq, 100, \num, 1]);
    250.do{|i|
        x.set(\freq, sin(0.2*i.pow(1.5))*100 + 200);
        0.02.wait;
    };
    x.free;
}.fork

* Try %scv and %%scv for verbose line resp. cells
* Try %scg and %%scg for getter line resp. cells

### Stop synths

In [ ]:
synth = "default"
%sc x = Synth.new(^synth)

* to stop all playing synths either use CMD-. (in Jupyter Command mode).
* It is a shortcut for the ´free_all´ method of the default server

In [ ]:
%sc s.freeAll  # leaves the s.scope running

In [ ]:
scn.SC.default.server.free_all()

which is also available using our `sc` instance directly

In [ ]:
sc.server.free_all()

## exit - startup - boot - delete

To shut down the server and sclang subprocesses and also end the OSC communication
one should use

In [ ]:
sc.exit()

If you use sc3nb in a notebook and restart the kernel the subprocesses will not be killed. This is an open issue of the Jupyter project: https://github.com/jupyter/jupyter_client/issues/104

`scn.startup` does allow to only boot the server

In [ ]:
sc = scn.startup(start_sclang=False)

You can produce a test sound with `blip`, which should relax any tensions whether sc is up and running. This sound should be played by the default server start.

In [ ]:
sc.server.blip()

The SC Server does include:
* a synthdef "s1" is created which is a discrete sound event with parameters 
    * frequency freq
    * duration dur
    * attack time att
    * amplitude amp
    * number of harmonics num
    * spatial panning pan
* synthdef "s2" is created which is a continuous synth with parameters
    * frequency freq
    * amplitude amp
    * number of harmonics num
    * spatial panning pan

* a synthdef "record" is created with parameter bufnum (buffer number), which simply records audio input to a buffer

if sclang is not started it will raise a runtime warning when accessing it

In [ ]:
sc.lang

In [ ]:
sc.start_sclang()

## scsynth control

### scsynth control with OSC

* Communication with scsynth is done via OSC using the cross-platform package python-osc.
* Direct control of synths shortcuts a detour via sclang and is both more efficient and promises a lower latency

* The basic wrapper for OSC messages, both to scsynth and sclang is 
`msg(msg_addr, msg_args, sclang=False)`
* By default, it sends messages to scsynth, but with sclang=True messages can be sent to sclangs OSC port.

In [ ]:
sc.server.msg("/s_new", ["s1", 1001, 1, 0, "freq", 300])  # a short 300 Hz tone

* note that you have to specify the nodeID (here 1001), possibly without knowing whether that nodeID is free. If it is not, the node could not be created.
* using nodeID -1 will let scsynth automatically select a free nodeID, however, so far there is no communication channel to receive this number from scsynth

In [ ]:
# a more complex example
for p in [0,2,4,7,5,5,9,7,7,12,11,12,7,4,0,2,4,5,7,9,7,5,4,2,4,0,-1,0,2,-5,-1,2,5,4,2,4]:
    freq = scn.midicps(60+p)  # see helper fns below
    sc.server.msg("/s_new", ["s1", -1, 1, 0, "freq", freq, "dur", 0.5, "num", 1])
    time.sleep(0.15)

* Note that the timing is here under python's control, which is not very precise.
* Bundles allow to specify a timetag and thus let scsynth control the timing, which is much better, if applicable.
* Bundles can be created by
        bundle(timetag, msg_addr, msg_args=None, sclang=False)

* Note that it is needed to call ´send´ on the bundle to send it.

In [ ]:
sc.server.bundler(1.2, "/s_new", ["s1", -1, 1, 0, "freq", 200, "dur", 1]).send()  # a tone starts in 1.2s
sc.server.bundler(1.7, "/s_new", ["s1", -1, 1, 0, "freq", 300, "dur", 1]).send()  # a tone starts in 1.7s

* small numbers are times in seconds relative to time.time() evaluated at fn execution
* use time.time()+timeoffset to specify in absolute times (see next example)

In [ ]:
t0 = time.time()
for i, r in enumerate(np.random.randn(100)):
    onset = t0 + 3 + r
    freq = 500 + 5 * i
    sc.server.bundler(onset, "/s_new", ["s1", -1, 1, 0, 
                                "freq", freq, "dur", 1.5, "num", abs(r)+1]).send()

**Remarks**:
* note that the python code returns immediately and all events remain in scsynth
* note that unfortunately scsynth has a limited buffer for OSC messages, so it is not viable to spawn thousends of events. scsynth will then simply reject OSC messages.
* this motivated (and is solved) with a TimedQueue, see below.

### scsynth control with SuperCollider Objects in Python

In [ ]:
syn = scn.Synth()

In [ ]:
syn.free()

## Recording sc3 output into a file with scsynth

In [ ]:
# use the Recording class to capture the output
recorder = scn.Recorder(path="my_recording.wav")

with sc.server.bundler() as bundler:
    recorder.start(0.1)
    # /s_new synth name, node id, add action (0 to head), target (1 default group), synth arguments...
    bundler.wait(0.2)
    scn.Synth("s1", {"freq": 200, "dur": 1})
    bundler.wait(0.3)
    scn.Synth("s1", {"freq": 300, "dur": 1})
    recorder.stop(1.5)

* note that the sorting in scsynth node tree is with 'at begin' rule
* otherwise the rendered tones would be rendered after the outbus was written to file resulting in an empty file.
* the file is located in the same folder as this .ipynb file.

## MIDI interface

## Helper functions

* SuperCollider coders are familiar and frequently use a number of useful converter functions
* the helper functions provide pythonic pendants namely currently for (to be extended):

### `linlin(x, x1, x2, y1, y2)`

* to linearly map x from between [x1, x2] to [y1, y2]
* non range check is done, and no clipping.
* for negative slope mapping, simply swap dmi and dma

In [ ]:
xs = np.linspace(1,9,100)
plt.plot(xs, [scn.linlin(x, 0, 10, 300, 500) for x in xs])
plt.axis([0,10,300,500]);

### `midicps` and `cpsmidi`

In [ ]:
scn.midicps(69.2)  # convert MIDI note to cycles per second (cps) in [Hz]

In [ ]:
scn.cpsmidi(440)   # and back to MIDI note (in float resolution)

### `clip(value, minimim, maximum)`

In [ ]:
xs = np.linspace(1,9,100)
plt.plot([scn.clip(x, 5, 7) for x in xs]);

### `ampdb` and `dbamp`

In [ ]:
# dbamp(db) converts dB value in amplitude, 0 dB = 1, '*2' \approx +6dB
dbs = np.linspace(-20, 20)
plt.plot(dbs, [scn.dbamp(d) for d in dbs]);
# plt.semilogy()

In [ ]:
# ampdb(amp) converts an amplitude to dB, assuming 0dB=1
scn.ampdb(0.2)

## Creating an OSC responder and msg to sclang for synthesis

In [ ]:
%%scv
OSCdef(\dinger, { | msg, time, addr, recvPort |
    var freq = msg[2];
    {Pulse.ar(freq, 0.04, 0.3)!2 * EnvGen.ar(Env.perc, doneAction:2)}.play()
}, '/ding')

In [ ]:
for i in range(100):
    sc.server.msg("/ding", ["freq", 1000-5*i], receiver=sc.lang.addr)

In [ ]:
%sc OSCdef.freeAll()

## TimedQueue Basics

Motivation:
* for sonifications with precise timing, thousends of events need to be spawn at the exact time.
* doing this with bundles doesn't work as the OSC buffer of scsynth is limited
* it needs a TimedQueue where events can be added for time-precise dispatching
* a thread then simply checks what items are due and executes them
* using arbitrary functions as objects for the queue allows to use it both for sonification (e.g. sending OSC messages/bundles) and even visualization
* however, the functions should complete really quickly as otherwise the queue would run late and fail to process due events
* hence, it is the responsibility of the user to be careful
* if, however, longer programs are needed, they can be spawn as threads on execution
* The following demos illustrate the core functionality with console print and sound

In [ ]:
queue = scn.TimedQueue()

In [ ]:
import sys
def myfun(x):
    os.write(1, "{}\n".format(x).encode())
    sys.stderr.flush()
    
def myblip(freq):
    sc.server.msg("/s_new", ["s1", -1, 1, 0, "freq", freq, "num", 3])

In [ ]:
myfun(4)

In [ ]:
myblip(700)

In [ ]:
t0 = time.time()
for i in range(50):
    queue.put(t0+i*0.04, myblip, 800+1*7*i)
    queue.put(t0+i*0.04, myfun,  400+30*i)  # plots on stderr = console
print(time.time()-t0)

In [ ]:
queue.close()

## TimedQueueSC

To allow easy and fast usage of SC messages and bundles `TimedQueueSC` was created
* `put_msg(onset, address, args, sclang=False)` allows to send a message from python at onset.
* `put_bundle(onset, timetag, address, args, sclang=False)` allows to send a bundle from python at onset.

## TimedQueueSC example with synchronized sound and mpl plot

* This example shows how to highlight data points as they are played.
* However, the marking is reset for every new data point, i.e. data points are not highlighted as long as the corresponding sound lasts
* to achieve that, see code example below

**Note that there are sometimes some strange effects with matplotlib event loop hickups in Mac, it is untested with Linux or Windows, any problem reports or suggested solutions are welcome.**

In [ ]:
import matplotlib
import matplotlib.pyplot as plt
%matplotlib qt5

In [ ]:
%matplotlib  qt5

In [ ]:
# load data
data = np.vstack((np.random.randn(50, 5), np.random.randn(100, 5)+3.5))

In [ ]:
# create figure, axis, plots
fig, ax = plt.subplots(1)  # create figure
mngr = plt.get_current_fig_manager(); mngr.window.setGeometry(1200, 0, 500, 400)
pldata, = ax.plot(data[:,1], data[:,2], ".", ms=5) # create plots
plmarked, = ax.plot([], [], "ro", ms=5, lw=0.5)
plt.show(block=False); plt.ion(); fig.canvas.draw() # not needed if plot shows

# create the queue
queue = scn.TimedQueueSC(sc)

In [ ]:
def update_plot(x, y):
    global fig, ax, pldata, plmarked
    plmarked.set_data([x], [y])
    ax.draw_artist(ax.patch)
    ax.draw_artist(pldata)
    ax.draw_artist(plmarked)
    fig.canvas.update() # additional fig.canvas.flush_events() not needed?

In [ ]:
t0 = time.time()
for i, r in enumerate(data):
    onset = t0 + scn.linlin(r[1], data[:,1].min(), data[:,1].max(), 0.1, 4) + random.random()*0.2 + 0.2
    freq = scn.midicps(scn.linlin(r[2], 2, 5, 60, 80))
    pos = scn.linlin(r[4], 0, 2, -1, 1)
    queue.put_bundle(onset-0.2, onset, "/s_new", ["s1", -1, 1, 0, "freq", freq, "amp", 0.05, "dur", .52, "pos", pos])
    queue.put(onset, update_plot, (r[1], r[2]), spawn=False)
print('time used:', time.time() - t0)

## TimedQueueSC PMSon with matplotlib highlights

* this example illustrates how to use TimedQueues to maintain a 'currently playing selection' of data points, so that the GUI highlight gets deactivated as the corresponding sound stops
* this is achieved by scheduling a select and unselect function invocation at the time stamps where the corresponding sound starts and ends
* Note that herre the actual plot update is in a second loop of scheduled 'update_plot' invocations at an independent user controlled frame rate

In [ ]:
data = np.vstack((np.random.randn(300, 7), np.random.randn(300, 7)+5))

In [ ]:
# create figure
fig, ax = plt.subplots(1)  # create figure
mngr = plt.get_current_fig_manager(); mngr.window.setGeometry(1200, 0, 500, 400)
plt.show()

# create the queue
queue = scn.TimedQueueSC(sc)

In [ ]:
def mapcol(row, stats, col, val_from, val_to):  # helper for mapping
    return scn.linlin(row[col], stats[col, 0], stats[col, 1], val_from, val_to)

def select(i):  #  highlight selection
    selected[i] = True

def unselect(i):
    selected[i] = False

def update_plot(xs, ys): 
    global fig, ax, pldata, plmarked, selected
    plmarked.set_data(xs[selected], ys[selected])
    ax.draw_artist(ax.patch)
    ax.draw_artist(pldata)
    ax.draw_artist(plmarked)
    fig.canvas.flush_events()
    fig.canvas.update()

# parameter mapping sonification with GUI
tot_dur = 5  # total duration of the sonification
max_ev_dur = 5.5  # maximal event duration
delay = 1  # offset

stats = np.vstack((np.min(data, 0), np.max(data, 0))).T
selected = np.zeros(np.shape(data)[0], np.bool)

# create axis, plots
ax.clear()
plmarked, = ax.plot([], [], "ro", ms=4, lw=0.5)
pldata, = ax.plot(data[:,1], data[:,2], ".", ms=2) # create plots

t0 = time.time()

for i, r in enumerate(data):
    onset = t0 + delay + 5* i/800 # mapcol(r, stats, 3, 0, tot_dur)
    freq  = scn.midicps( mapcol(r, stats, 2, 60, 90))
    ev_dur = mapcol(r, stats, 4, 0.2, max_ev_dur)
    # sonification
    synth_args = ["s1", -1, 1, 0, "freq", freq, "amp", 0.05, "dur", ev_dur, "pos", pos]
    queue.put_bundle(onset-delay, onset, "/s_new", synth_args)
    # on/off events of marker highlight
    queue.put(onset, select, i)
    queue.put(onset + ev_dur, unselect, i)

# update plot at given rate from earliest to latext time
for t in np.arange(t0, t0+delay+tot_dur+ev_dur+1, 1/10):  # 1 / update rate
    queue.put(t, update_plot, (data[:,1], data[:,2]))

## TimedQueueSC PMSon with timeseries data and matplotlib

* The following example illustrates howto create a continuous sonification with concurrent plotting the time in a plot
* This presumes time-indexable data
* a maximum onset variable is maintained to shutdown the continuously playing synths when done
* note that the highlight will only replot the marker, required time is thus independent of the amount of data plotted in the other plot.

In [ ]:
ts = np.arange(0, 20, 0.01)
data = np.vstack((ts, 
                  np.sin(2.5*ts) + 0.01*ts*np.random.randn(np.shape(ts)[0]), 
                  0.08*ts[::-1]*np.cos(3.5*ts)**2)).T

In [ ]:
# create figure
fig, ax = plt.subplots(1)  # create figure
mngr = plt.get_current_fig_manager(); mngr.window.setGeometry(1200, 0, 500, 400)

# create axis, plots
ax.clear()
plmarked, = ax.plot([], [], "r-", lw=1)
pldata, = ax.plot(data[:,0], data[:,1], "-", ms=2) # create plots
pldataR, = ax.plot(data[:,0], data[:,2], "-", ms=2) # create plots

In [ ]:
# create the queue
queue = scn.TimedQueueSC(sc)

def mapcol(row, stats, col, val_from, val_to):  # helper for mapping
    return scn.linlin(row[col], stats[col, 0], stats[col, 1], val_from, val_to)

def update_plot(t): 
    global fig, ax, pldata, plmarked, selected
    plmarked.set_data([t,t], [-10000, 10000])
    ax.draw_artist(ax.patch)
    ax.draw_artist(pldata)
    ax.draw_artist(pldataR)
    ax.draw_artist(plmarked)
    fig.canvas.update()
    # fig.canvas.flush_events()

stats = np.vstack((np.min(data, 0), np.max(data, 0))).T
selected = np.zeros(np.shape(data)[0], np.bool)

# parameter mapping sonification with GUI
delay = 0.5
rate = 2

t0 = time.time()
queue.put_msg(t0, "/s_new", ["s2", 1200, 1, 0, "amp", 0])
queue.put_msg(t0, "/s_new", ["s2", 1201, 1, 0, "amp", 0])

max_onset = 0
latest_gui_onset = 0
gui_frame_rate = 60

ts = []
for i, r in enumerate(data[::2, :]):
    ts.append(time.time()-t0)
    if i==0: tmin = r[0]
    onset = (r[0]-tmin)/rate
    freq  = scn.midicps( mapcol(r, stats, 1, 60, 70))
    freqR = 0.5 * scn.midicps( mapcol(r, stats, 2, 70, 80))

    # sonification
    tt = t0 + delay + onset
    if tt > max_onset: max_onset = tt
    queue.put_bundle(tt-0.2, tt, "/n_set", 
        [1200, "freq", freq, "num", 4, "amp", 0.2, "pan", -1, "lg", 0])
    queue.put_bundle(tt-0.2, tt, "/n_set", 
        [1201, "freq", freqR, "num", 1, "amp", 0.1, "pan", 1])
    if tt > latest_gui_onset + 1/gui_frame_rate:  # not more than needed gui updates
        latest_gui_onset = tt
        queue.put(tt, update_plot, (r[0],), spawn=False)
queue.put(max_onset, sc.server.msg, ("/n_free", 1200))
queue.put(max_onset, sc.server.msg, ("/n_free", 1201))
              
# queue.join()
print(time.time()-t0)

In [ ]:
sc.server.bundler(0, "/s_new", ["s2", 1200, 1, 0, "amp", 0.04]).send()
sc.server.bundler(0.2, "/n_set", [1200, "freq", 200, "num", 1, "amp", 0.2, "pan", 0]).send()
sc.server.bundler(0.4, "/n_free", 1200).send()

In [ ]:
sc.exit()